In [1]:
import concurrent.futures
import requests
import os
import pandas as pd
from tqdm import tqdm


In [8]:
SOURCE_SWISSIMAGE = './source/ch.swisstopo.swissimage-dop10-dZ7D5lPH.csv'
SOURCE_SWISSIMAGE_SAVE_DIR = './dataset/base'

SOURCE_SWISSALTI3D = './source/ch.swisstopo.swissalti3d-IMqs1b4h.csv'
SOURCE_SWISSALTI3D_SAVE_DIR = './dataset/alti'

In [3]:
df = pd.read_csv(SOURCE_SWISSIMAGE, names=['url'])
df_alit = pd.read_csv(SOURCE_SWISSALTI3D, names=['url'])

In [9]:
def download_image(url, save_dir):
    try:
        filename = os.path.join(save_dir, url.split("/")[-1])

        if os.path.isfile(filename):
            return

        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(filename, 'wb') as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)
        else:
            print(f"Failed to download: {url}, Status Code: {response.status_code}")
    except Exception as e:
        print(f"Error downloading {url}: {e}")

def download_images(urls, save_dir, max_threads=10):
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_threads) as executor:
        list(tqdm(executor.map(lambda x: download_image(x, save_dir), urls), total=len(urls)))
        

In [ ]:
download_images(list(df['url']), SOURCE_SWISSIMAGE_SAVE_DIR, max_threads=10)
download_images(list(df_alit['url']), SOURCE_SWISSALTI3D_SAVE_DIR, max_threads=10)